# Coronagraph simulmation


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D   # this is for surface plots
from matplotlib import cm
import matplotlib.units as units 
from PIL import Image, ImageDraw
from astropy.io import fits
%matplotlib inline

Opitical Parameters

All units are in SI!!!!

In [ ]:
pupil_center = (0,0,0)
diameter=1

focal_center = (1,0,0)
focal=1

lambda_center = 0.45e-6
lambda_wide = 0.15e-6
lambda_resolution = lambda_wide/10

power=1e-3 #W/m²


def physic_converter():
    meter = npix*diameter/(2*diameter_comp)
    meter2 = meter * PAD
    radian = lambda_center/(meter*PAD)
    second = 180*radian/(np.pi*3600)
    return meter, meter2, radian, second

Useful TF functions

In [ ]:
PAD=4 #sampling size, min=2 SHANON

def ft2d(func):
    ft = np.fft.fftshift(np.fft.fft2(np.fft.ifftshift(func)))
    return ft

def ift2d(func):
    ift = np.fft.fftshift(np.fft.ifft2(np.fft.ifftshift(func)))
    return ift

def ft1d_freq(x):
    """Calculate the (spatial) frequency array based on the spatial array x."""
    s = np.fft.fftshift(np.fft.fftfreq(x.size, d = x[-1]-x[-2]))
    return s

def padcplx(c):
    """Puts a Complex array in the centre of a zero-filled Complex array.
    PAD defines the padding multiplier for the output array."""
    (nx, ny) = c.shape
    bignx = nx * PAD + 1
    bigny = ny * PAD + 1
    big_c = np.zeros((bignx, bigny), dtype = complex)
    
    dx = int((nx * (PAD-1)) / 2 + 1)
    dy = int((ny * (PAD-1)) / 2 + 1)
    
    big_c[dx:dx+nx,dy:dy+ny] = c
    return(big_c)

In [ ]:
#Plotting function!

def multi_plot(func, title, rescale, axe_name, loga):
    #Some fancy stuff  
    plt.figure(figsize = (50, 10))
    plt.axis('equal')
    font = {' - size'   : 50}

    plt.subplot(1, 5, 1)
    plt.imshow(np.abs(func),extent=[-rescale,rescale,-rescale,rescale])
    plt.xlabel(axe_name)
    plt.ylabel(axe_name)
    plt.colorbar()
    plt.title(title+' - Amplitude')

    plt.subplot(1, 5, 2)
    if(loga == 1):
        plt.imshow(np.log10(np.abs(func)**2),extent=[-rescale,rescale,-rescale,rescale])
        plt.xlabel(axe_name)
        plt.ylabel(axe_name)
        plt.colorbar()
        plt.title(title+' - log(I) aka log(PSF)')
    else:
        plt.imshow(np.abs(func)**2,extent=[-rescale,rescale,-rescale,rescale])
        plt.xlabel(axe_name)
        plt.ylabel(axe_name)
        plt.colorbar()
        plt.title(title+' - I aka PSF')
        
    plt.subplot(1, 5, 3)
    plt.imshow(np.real(func),cmap = 'RdBu',extent=[-rescale,rescale,-rescale,rescale])
    plt.xlabel(axe_name)
    plt.ylabel(axe_name)
    plt.colorbar()
    plt.title(title+' - Real part')

    plt.subplot(1, 5, 4)
    plt.imshow(np.imag(func),cmap = 'RdBu',extent=[-rescale,rescale,-rescale,rescale])
    plt.xlabel(axe_name)
    plt.ylabel(axe_name)
    plt.colorbar()
    plt.title(title+' - Imaginary part')

    plt.subplot(1, 5, 5)
    plt.imshow(np.angle(func) ,cmap = 'PiYG',extent=[-rescale,rescale,-rescale,rescale])
    plt.xlabel(axe_name)
    plt.ylabel(axe_name)
    plt.colorbar(label='rads PtV')
    plt.title(title+' - Phase')
    return()

def zoom(im, zoomfac):
    """Cut out a square box from image im centered on (x,y) with half-box size bb."""
    #Box rezize
    x = int(im.shape[0]/2)
    y = int(im.shape[0]/2)
    npix_pad = npix*PAD+1   # figure out the padded big array sizes after the FT
    box = int(npix_pad/zoomfac)

    return(im[y-box:y+box,x-box:x+box])

Pupile plan

1 Form

2 Amplitude

3 Phase

In [ ]:
# Define a function for a circular aperture
def circle_mask(im, xc, yc, rad):
    """Create a circular aperture centered on (xc, yc) with radius rcirc."""
    x, y = np.shape(im)
    newy, newx = np.mgrid[:y,:x]
    circ = (newx-xc)**2 + (newy-yc)**2 < rad**2
    return circ.astype('float')


def hexagone_mask(im, xc, yc):
#https://stackoverflow.com/posts/3732128/edit
    x, y = np.shape(im)
    newy, newx = np.mgrid[:y,:x]
    hypothenus = npix/4
    diameter_comp = hypothenus
    
    x1 = xc+hypothenus
    x2 = xc+0.5*hypothenus
    x3 = xc-0.5*hypothenus
    x4 = xc-h
    x5 = xc-0.5*hypothenus
    x6 = xc+0.5*hypothenus
    y1 = yc
    y2 = yc+hypothenus*np.sqrt(3/2)
    y3 = yc+hypothenus*np.sqrt(3/2)
    y4 = yc
    y5 = yc-hypothenus*np.sqrt(3/2)
    y6 = yc-hypothenus*np.sqrt(3/2)
      
    polygon = [(x1,y1),(x2,y2),(x3,y3),(x4,y4),(x5,y5),(x6,y6)]
    width = npix
    height = npix

    #Create image and convert to array
    img = Image.new('L', (width, height), 0)
    ImageDraw.Draw(img).polygon(polygon, outline = 1, fill = 1)
    img = grid*img

    return img


#Amplitude
#Grids
npix = 501 #FFT need a centering grid put a ODD
grid = np.full(shape = (npix,npix),fill_value = 1)


rad = 0.7 * npix/2
diameter_comp = rad*2
aperture= circle_mask(grid, int(npix/2), int(npix/2), rad)
#aperture = hexagone_mask(grid, int(npix/2), int(npix/2))

#aperture = fits.getdata('../../JWST_pupil.fits')
#npix = len(aperture)
#grid = np.full(shape = (npix,npix),fill_value = 1) #JWST


meter_scale, meter_scale2, radian_scale, second_scale = physic_converter()

A_star = 1
A_planet = 10e-10

plt.xlabel("meter")
plt.ylabel("meter")
plt.imshow(aperture,extent=[-meter_scale,meter_scale,-meter_scale,meter_scale])
plt.colorbar()
plt.title('Aperture')

In [ ]:
def zernike (zernikecoef, xc, yc, grid, norma):
    phase = grid*0
    
    lin2 = np.linspace(0-xc, npix-xc, npix)
    xx, yy = np.meshgrid(lin2, lin2)

    rho = np.sqrt(xx**2+yy**2)
    rho /= norma
    theta = -np.arctan2(yy,-xx)
    
    #TODO Generalizeit!
    Z = [1,
       2*rho*np.cos(theta),2*rho*np.sin(theta),
       np.sqrt(6)*rho**2*np.cos(2*theta) ,np.sqrt(3)*(rho**2-1) ,np.sqrt(6)*rho**2*np.sin(2*theta)]
    
    for i in range (len(zernikecoef)):
        phase = phase + zernikecoef[i] * Z[i]

    return phase
    
zernikecoef_star = [0,0,0,0,0,0]
zernikecoef_planet = [0,0,2,0,0,0]

xc = int(npix/2)+npix%2
yc = xc
rad = len(aperture)

phi_star = zernike(zernikecoef_star, xc, yc, grid, npix)*aperture
phi_planet = zernike(zernikecoef_planet, xc, yc, grid, npix)*aperture

plt.figure(figsize = (20, 10))
plt.axis('equal')
font = {' - size'   : 50}

plt.subplot(1, 2, 1)
plt.xlabel("meter")
plt.ylabel("meter")
plt.imshow(phi_star,cmap = 'PiYG',extent=[-meter_scale,meter_scale,-meter_scale,meter_scale])
plt.colorbar(label='rads PtV')
plt.title('star phase')

plt.subplot(1, 2, 2)
plt.xlabel("meter")
plt.ylabel("meter")
plt.imshow(phi_planet,cmap = 'PiYG',extent=[-meter_scale,meter_scale,-meter_scale,meter_scale])
plt.colorbar(label='rads PtV')
plt.title('planet phase')

In [ ]:
#Electrical Field afetr aperture
def Efield(Amplitude, phi, plot):
    e_field = Amplitude*aperture*np.exp(1j*meter_scale*phi/lambda_center)
    
    if(plot==1):
        multi_plot(e_field, "PP Eletric field", meter_scale, "meter" ,loga=0)
    
    return e_field

Efield_star = Efield(A_star, phi_star, 1)
Efield_planet = Efield(A_planet, phi_planet, 1)

Focal plan

1 TF aperture

2 Make coronaph aperture

3 Multiply TF and coronagraph

In [ ]:
def first_lens(e_field, plot):
    focalplan_BC = ft2d(padcplx(e_field))
    
    if(plot==1):
        focalplan_BC_zoom = zoom(focalplan_BC, 10)
        multi_plot(focalplan_BC_zoom, "FP", radian_scale ,"rads" ,loga = 1)
    return focalplan_BC

focalplan_BC_star = first_lens(Efield_star, 1)
focalplan_BC_planet = first_lens(Efield_planet, 1)

#Coronagraph

FPM

LyotStop plan

1 TF focal pla after Conagraph

2 Make (Lyotstop) aperture

3 Multiply TF and (Lyotstop) apertureLyotStop plan


In [ ]:
#Clasic Lyot Coronagraph (CLC)

def CLC(focalplan_BC, FPM_rad, Lyot_rad, plot):
    FPM = 1 - circle_mask(focalplan_BC, int(focalplan_BC.shape[0]/2), int(focalplan_BC.shape[0]/2), FPM_rad)
    focalplan_AC = focalplan_BC * FPM
    B_Lyot = ft2d(focalplan_AC)

    Lyotstop = circle_mask(B_Lyot, int(B_Lyot.shape[0]/2), int(B_Lyot.shape[0]/2), Lyot_rad)
    
    if(plot==1):
        multi_plot(focalplan_AC, "FP AC", radian_scale ,"lambda/D",loga = 0)
        B_Lyot_zoom = zoom(B_Lyot, 5)
        multi_plot(B_Lyot_zoom, "LP BL", meter_scale2 ,"lambda/D",loga = 0) 
    return (B_Lyot * Lyotstop)


def last_lens(A_Lyot, plot):
    screen = ft2d(A_Lyot)

    if(plot==1):
        A_Lyot_zoom = zoom(A_Lyot, 5)
        screen_zoom = zoom(screen, 2)
        multi_plot(A_Lyot_zoom, "LP AL", meter_scale2 ,"lambda/D" ,0)
        multi_plot(screen_zoom, "Screen", radian_scale ,"lambda/D",1)
    return screen


A_Lyot_star = CLC(focalplan_BC_star, rad*0.9, rad*0.98, 1)
A_Lyot_planet = CLC(focalplan_BC_planet, rad*0.9, rad*0.98, 1)

screen_star = last_lens(A_Lyot_star, 1)
screen_planet = last_lens(A_Lyot_planet, 1)

In [ ]:
def full_coronagraph_glasses(Amplitude, phi, FPM_rad, Lyot_rad, plot):
    A = Efield(Amplitude, phi, plot)
    B = first_lens(A, plot)
    C = CLC(B , FPM_rad, Lyot_rad, plot)
    return (last_lens(C, plot))

full_coronagraph_glasses(A_star, phi_star, rad*0.9, rad*0.98, 1)

In [ ]:
diff=ft2d(ft2d(focalplan_BC))-screen

diff_zoom = zoom(screen, 4)

multi_plot(diff_zoom, "difference", radian_scale ,"lambda/D",1)

In [ ]:
#FINAL ploting Cell
plt.figure(figsize = (20, 60))
#plt.axis('equal')
#plt.subplots_adjust(wspace = 0.5)

font = {' - size'   : 15}

"""
multi_plot(focalplan_BC, "FP BC",1)
multi_plot(focalplan_AC, "FP AC",1)
multi_plot(B_Lyot, "LP BL",1)
multi_plot(A_Lyot, "LP AL",1)
multi_plot(screen, "Screen",1)
"""

func2 = [Efield,focalplan_BC,focalplan_AC,B_Lyot,A_Lyot,screen]
#func2 = [Efield,focalplan_BC]
title2 = ["PP Eletric field","FP BC","FP AC","LP BL","LP AL","Screen"]

for i in range(6):
    plt.subplot(30, 5, i*5+1)
    plt.imshow(np.abs(func2[i]))
    plt.colorbar()
    plt.title(title2[i]+' - Amplitude')

    plt.subplot(30, 5, i*5+2)
    plt.imshow(np.log(np.abs(func2[i])**2))
    plt.colorbar()
    plt.title(title2[i]+' - log(I) aka log(PSF)')

    plt.subplot(30, 5, i*5+3)
    plt.imshow(np.real(func2[i]),cmap = 'RdBu')
    plt.colorbar()
    plt.title(title2[i]+' - Real part')

    plt.subplot(30, 5, i*5+4)
    plt.imshow(np.imag(func2[i]),cmap = 'RdBu')
    plt.colorbar()
    plt.title(title2[i]+' - Imaginary part')

    plt.subplot(30, 5, i*5+5)
    plt.imshow(np.angle(func2[i]) ,cmap = 'PiYG')
    plt.colorbar()
    plt.title(title2[i]+' - Phase')

print("PP = Pupil Plan, FP = Focal Plan, BC = Before Coronagraph, AC = Before Coronagraph, LP = LyotStop Plan, I = Intensity")